# Robust Phishing URL Detection under Adversarial Attacks

## Objective
本PoCではフィッシングURL検知モデルに対し、

- 通常性能評価
- Evasion Attack（回避攻撃）
- Poisoning Attack（学習汚染）
- 防御（Adversarial Training）

を実施し、

「精度」ではなく
**"実運用で安全に使えるロバスト性"** を評価する。

SOC/CSIRT環境での実装を想定した実務寄り検証を目的とする。



# Threat Model

## System Overview
URLフィルタリング/メールゲートウェイにおいて、
アクセス先URLがフィッシングサイトか否かを分類するML検知器を想定。

## Attacker Goal
- フィッシングサイトへ誘導成功
- 検知回避によるブロック回避

## Attacker Capability
攻撃者は以下を改変可能と仮定：
- URL長の調整（短縮URL）
- HTTPS偽装
- iframe削除
- ドメイン偽装
- 一部学習データの汚染

## Defender Goal
- 検知漏れ（False Negative）最小化
- 誤検知抑制
- SOC運用可能な安定性能維持

## Evaluation Metrics
- Recall（主指標）
- 攻撃前後の性能劣化率
- ロバスト性


In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, ConfusionMatrixDisplay,
    roc_curve, auc
)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

import shap
import optuna

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)



In [48]:
training_data = np.genfromtxt('dataset.csv', delimiter=',', dtype=np.int32)
df = pd.DataFrame(training_data)

In [49]:
# Create feature names for the dataset
feature_names = [
    'having_ip_address',
    'url_length',
    'shortining_service',
    'having_at_symbol',
    'double_slash_redirecting',
    'prefix_suffix',
    'having_sub_domain',
    'sslfinal_state',
    'domain_registration_length',
    'favicon',
    'port',
    'https_token',
    'request_url',
    'url_of_anchor',
    'links_in_tags',
    'sfh',
    'submitting_to_email',
    'abnormal_url',
    'redirect',
    'on_mouseover',
    'rightclick',
    'popupwindow',
    'iframe',
    'age_of_domain',
    'dnsrecord',
    'web_traffic',
    'page_rank',
    'google_index',
    'links_pointing_to_page',
    'statistical_report',
    'label'  # Target variable
]

# Add column names to the dataframe
df.columns = feature_names
df.head()

,having_ip_address,url_length,shortining_service,having_at_symbol,double_slash_redirecting,prefix_suffix,having_sub_domain,sslfinal_state,domain_registration_length,favicon,...,popupwindow,iframe,age_of_domain,dnsrecord,web_traffic,page_rank,google_index,links_pointing_to_page,statistical_report,label
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1


# フィッシング検出データセットの説明

dataset_info = """
## フィッシング検出用データセット概要

### データセット名
Phishing Websites Dataset

### 寄贈日
2015年3月25日

### データ収集元
- PhishTank archive
- MillerSmiles archive
- Googleの検索オペレータ

### データセット特性
- **形式**: タビュラー（表形式）
- **対象分野**: コンピュータサイエンス
- **関連タスク**: 分類問題
- **特徴量タイプ**: 整数値
- **インスタンス数**: 11,055件
- **特徴量数**: 30個
- **欠損値**: なし

### 重要な注記
本研究の課題の1つは、信頼できる訓練データセットの入手が困難なことであった。
フィッシングウェブサイト予測に関する多くの論文が発表されているにもかかわらず、
公開された信頼できる訓練データセットがほとんど存在しない理由は、
フィッシングウェブページの特性を定義する特徴について学術文献での合意がないためである。

本データセットでは、フィッシングウェブサイトの予測に有効であることが証明された重要な特徴量と、
新たに提案した特徴量を明らかにしている。

### 30個の特徴量一覧
1. having_ip_address（IPアドレス保有の有無）
2. url_length（URL長）
3. shortining_service（短縮サービス利用の有無）
4. having_at_symbol（@シンボルの保有の有無）
5. double_slash_redirecting（ダブルスラッシュリダイレクト）
6. prefix_suffix（プリフィックス・サフィックス）
7. having_sub_domain（サブドメイン保有の有無）
8. sslfinal_state（SSL最終状態）
9. domain_registration_length（ドメイン登録期間）
10. favicon（ファビコン）
11. port（ポート）
12. https_token（HTTPSトークン）
13. request_url（リクエストURL）
14. url_of_anchor（アンカーURL）
15. links_in_tags（タグ内のリンク）
16. sfh（サーバーフォーム処理）
17. submitting_to_email（メール送信）
18. abnormal_url（異常URL）
19. redirect（リダイレクト）
20. on_mouseover（マウスオーバー）
21. rightclick（右クリック）
22. popupwindow（ポップアップウィンドウ）
23. iframe（iframe）
24. age_of_domain（ドメイン年齢）
25. dnsrecord（DNSレコード）
26. web_traffic（ウェブトラフィック）
27. page_rank（ページランク）
28. google_index（Google インデックス）
29. links_pointing_to_page（ページへのリンク数）
30. statistical_report（統計レポート）

### 参照論文
- R. Mohammad, F. Thabtah, L. Mccluskey (2012)
- 論文題: "An assessment of features related to phishing websites using an automated technique"
- 出版: International Conference for Internet Technology and Secured Transactions
"""

In [50]:
X = df.drop("label", axis=1)
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE
)

print(X.shape, y.shape)

(11055, 30) (11055,)


In [51]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate(y_true, y_pred, title):

    print(f"\n==== {title} ====")

    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average="binary", pos_label=1, zero_division=0))
    print("Recall   :", recall_score(y_true, y_pred, average="binary", pos_label=1, zero_division=0))
    print("F1       :", f1_score(y_true, y_pred, average="binary", pos_label=1, zero_division=0))


In [52]:
import optuna
from sklearn.model_selection import cross_val_score

def objective_lr(trial):

    C = trial.suggest_float("C", 1e-3, 10, log=True)

    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            C=C,
            max_iter=3000,
            class_weight="balanced",
            random_state=RANDOM_STATE
        ))
    ])

    score = cross_val_score(
        pipe,
        X_train,
        y_train,
        cv=5,
        scoring="f1"
    ).mean()

    return score


study_lr = optuna.create_study(direction="maximize")
study_lr.optimize(objective_lr, n_trials=30)

print("Best params:", study_lr.best_params)


[I 2026-02-12 01:30:57,954] A new study created in memory with name: no-name-95d143c9-a50a-4302-8fd8-082ee227423a
[I 2026-02-12 01:30:58,033] Trial 0 finished with value: 0.934047879205236 and parameters: {'C': 0.01435892228092246}. Best is trial 0 with value: 0.934047879205236.
[I 2026-02-12 01:30:58,109] Trial 1 finished with value: 0.9342496088388345 and parameters: {'C': 0.019492039807672834}. Best is trial 1 with value: 0.9342496088388345.
[I 2026-02-12 01:30:58,178] Trial 2 finished with value: 0.9327782461029217 and parameters: {'C': 0.002038889484726509}. Best is trial 1 with value: 0.9342496088388345.
[I 2026-02-12 01:30:58,341] Trial 3 finished with value: 0.9332912157603914 and parameters: {'C': 3.956603774332178}. Best is trial 1 with value: 0.9342496088388345.
[I 2026-02-12 01:30:58,461] Trial 4 finished with value: 0.9332912157603914 and parameters: {'C': 4.631539181592398}. Best is trial 1 with value: 0.9342496088388345.
[I 2026-02-12 01:30:58,539] Trial 5 finished with 

Best params: {'C': 0.006476932788045012}


In [53]:
best_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        **study_lr.best_params,
        max_iter=3000,
        class_weight="balanced",
        random_state=RANDOM_STATE
    ))
])

best_lr.fit(X_train, y_train)

pred_lr = best_lr.predict(X_test)

evaluate(y_test, pred_lr, "Tuned Logistic Regression")



==== Tuned Logistic Regression ====
Accuracy : 0.9289914066033469
Precision: 0.9309791332263242
Recall   : 0.942323314378554
F1       : 0.9366168752523214


In [54]:
def objective_tree(trial):

    max_depth = trial.suggest_int("max_depth", 2, 12)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)

    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        class_weight="balanced",
        random_state=RANDOM_STATE
    )

    score = cross_val_score(
        model,
        X_train,
        y_train,
        cv=5,
        scoring="f1"
    ).mean()

    return score


study_tree = optuna.create_study(direction="maximize")
study_tree.optimize(objective_tree, n_trials=30)

print("Best params:", study_tree.best_params)


[I 2026-02-12 01:31:00,277] A new study created in memory with name: no-name-c0e77d80-5ad8-4988-a590-66270d5f11f8
[I 2026-02-12 01:31:00,313] Trial 0 finished with value: 0.9145901053868826 and parameters: {'max_depth': 2, 'min_samples_split': 9}. Best is trial 0 with value: 0.9145901053868826.
[I 2026-02-12 01:31:00,364] Trial 1 finished with value: 0.939522971437279 and parameters: {'max_depth': 7, 'min_samples_split': 5}. Best is trial 1 with value: 0.939522971437279.
[I 2026-02-12 01:31:00,416] Trial 2 finished with value: 0.942664790976971 and parameters: {'max_depth': 8, 'min_samples_split': 10}. Best is trial 2 with value: 0.942664790976971.
[I 2026-02-12 01:31:00,472] Trial 3 finished with value: 0.9493766034956395 and parameters: {'max_depth': 10, 'min_samples_split': 8}. Best is trial 3 with value: 0.9493766034956395.
[I 2026-02-12 01:31:00,524] Trial 4 finished with value: 0.939522971437279 and parameters: {'max_depth': 7, 'min_samples_split': 5}. Best is trial 3 with value:

Best params: {'max_depth': 12, 'min_samples_split': 14}


In [55]:
tree_pipe = Pipeline([
    ("clf", DecisionTreeClassifier(
        max_depth=5,
        class_weight="balanced",
        random_state=RANDOM_STATE
    ))
])

tree_pipe.fit(X_train, y_train)

pred_tree = tree_pipe.predict(X_test)

evaluate(y_test, pred_tree, "Decision Tree Baseline")



==== Decision Tree Baseline ====
Accuracy : 0.9226594301221167
Precision: 0.8984962406015038
Recall   : 0.9707554833468724
F1       : 0.9332292073408824


In [56]:
def objective(trial):
    depth = trial.suggest_int("max_depth", 3, 15)

    model = DecisionTreeClassifier(max_depth=depth)

    score = cross_val_score(model, X_train, y_train, cv=5, scoring="recall").mean()
    return score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

best_depth = study.best_params["max_depth"]
print("Best depth:", best_depth)


[I 2026-02-12 01:31:01,958] A new study created in memory with name: no-name-5027083e-6582-48dd-8c9a-df7304c37caa
[I 2026-02-12 01:31:02,012] Trial 0 finished with value: 0.9411286951328754 and parameters: {'max_depth': 8}. Best is trial 0 with value: 0.9411286951328754.
[I 2026-02-12 01:31:02,069] Trial 1 finished with value: 0.9642728143243995 and parameters: {'max_depth': 15}. Best is trial 1 with value: 0.9642728143243995.
[I 2026-02-12 01:31:02,106] Trial 2 finished with value: 0.9522988848961604 and parameters: {'max_depth': 4}. Best is trial 1 with value: 0.9642728143243995.
[I 2026-02-12 01:31:02,143] Trial 3 finished with value: 0.9522988848961604 and parameters: {'max_depth': 4}. Best is trial 1 with value: 0.9642728143243995.
[I 2026-02-12 01:31:02,198] Trial 4 finished with value: 0.9527039466232843 and parameters: {'max_depth': 11}. Best is trial 1 with value: 0.9642728143243995.
[I 2026-02-12 01:31:02,269] Trial 5 finished with value: 0.9626490666282267 and parameters: {'

Best depth: 15


In [57]:
def objective_tree(trial):

    params = {
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10)
    }

    model = DecisionTreeClassifier(**params, random_state=42)

    model.fit(X_train, y_train)

    pred = model.predict(X_test)

    return f1_score(y_test, pred, average="binary", pos_label=1)


study_tree = optuna.create_study(direction="maximize")
study_tree.optimize(objective_tree, n_trials=30)

best_tree = DecisionTreeClassifier(**study_tree.best_params, random_state=42)
best_tree.fit(X_train, y_train)

print("Best Tree params:", study_tree.best_params)


[I 2026-02-12 01:31:03,051] A new study created in memory with name: no-name-a7bbdfab-bbd1-45b8-b8fe-fb482ec8ebb7
[I 2026-02-12 01:31:03,063] Trial 0 finished with value: 0.9397005261027924 and parameters: {'max_depth': 7, 'min_samples_split': 16, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.9397005261027924.
[I 2026-02-12 01:31:03,077] Trial 1 finished with value: 0.9604838709677419 and parameters: {'max_depth': 18, 'min_samples_split': 10, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.9604838709677419.
[I 2026-02-12 01:31:03,090] Trial 2 finished with value: 0.9438754608766898 and parameters: {'max_depth': 8, 'min_samples_split': 19, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.9604838709677419.
[I 2026-02-12 01:31:03,104] Trial 3 finished with value: 0.954083705810646 and parameters: {'max_depth': 20, 'min_samples_split': 3, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.9604838709677419.
[I 2026-02-12 01:31:03,118] Trial 4 finished with value: 0.943

Best Tree params: {'max_depth': 17, 'min_samples_split': 9, 'min_samples_leaf': 1}


In [58]:
def evasion_attack(model, X, top_k=5, noise_scale=2.0):
    """
    SHAP重要特徴に対する摂動ベースのEvasion攻撃シミュレーション。
    モデルの耐攻撃性評価に使用。
    """

    X_adv = X.copy()

    # SHAPで重要特徴抽出
    explainer = shap.Explainer(model.named_steps["clf"], X)
    shap_values = explainer(X[:500])

    importance = np.abs(shap_values.values).mean(axis=0)
    top_idx = np.argsort(importance)[-top_k:]

    for idx in top_idx:
        col = X.columns[idx]

        noise = np.random.normal(
            0,
            noise_scale * X[col].std(),
            size=len(X_adv)
        )

        # 攻撃方向（符号反転）
        X_adv[col] = X_adv[col] - noise

    return X_adv


In [59]:
X_adv = evasion_attack(best_lr, X_test, top_k=6, noise_scale=2.5)

print("=== Before attack ===")
evaluate(y_test, best_lr.predict(X_test), "LR Clean")

print("=== After attack ===")
evaluate(y_test, best_lr.predict(X_adv), "LR Attacked")


=== Before attack ===

==== LR Clean ====
Accuracy : 0.9289914066033469
Precision: 0.9309791332263242
Recall   : 0.942323314378554
F1       : 0.9366168752523214
=== After attack ===

==== LR Attacked ====
Accuracy : 0.6956128448665763
Precision: 0.7401032702237521
Recall   : 0.6986190089358245
F1       : 0.7187630589218554


In [60]:
flip_ratio = 0.1

y_poison = y_train.copy().reset_index(drop=True)
X_poison = X_train.copy().reset_index(drop=True)

np.random.seed(42)

flip_idx = np.random.choice(
    len(y_poison),
    int(len(y_poison) * flip_ratio),
    replace=False
)

# ⭐ 正しいラベル反転（±1専用）
y_poison.iloc[flip_idx] = -y_poison.iloc[flip_idx]

print(f"Flipped samples: {len(flip_idx)}")

best_lr.fit(X_poison, y_poison)

pred_poison = best_lr.predict(X_test)

evaluate(y_test, pred_poison, "After Poisoning")


Flipped samples: 884

==== After Poisoning ====
Accuracy : 0.9258254183627318
Precision: 0.9326845093268451
Recall   : 0.934199837530463
F1       : 0.9334415584415584


In [61]:
X_aug = pd.concat([
    X_train,
    evasion_attack(best_lr, X_train)
], ignore_index=True)

y_aug = pd.concat([
    y_train,
    y_train
], ignore_index=True)

best_lr.fit(X_aug, y_aug)

evaluate(y_test, best_lr.predict(X_adv), "After Defense")



==== After Defense ====
Accuracy : 0.6625961103573044
Precision: 0.7056827820186599
Recall   : 0.6758732737611698
F1       : 0.6904564315352697


In [62]:
from sklearn.metrics import precision_score, recall_score, f1_score

models = {
    "Logistic": best_lr,
    "DecisionTree": best_tree
}

rows = []

for name, model in models.items():

    pred_clean = model.predict(X_test)
    pred_adv   = model.predict(X_adv)

    rows.append({
        "Model": name,
        "Recall Clean": recall_score(y_test, pred_clean),
        "Recall Attack": recall_score(y_test, pred_adv),
        "F1 Clean": f1_score(y_test, pred_clean),
        "F1 Attack": f1_score(y_test, pred_adv)
    })

pd.DataFrame(rows)


,Model,Recall Clean,Recall Attack,F1 Clean,F1 Attack
0,Logistic,0.876523,0.675873,0.898418,0.690456
1,DecisionTree,0.969131,0.622258,0.964820,0.664931


# Operational Discussion

本検証から、MLベースの検知器は通常精度が高くても、
攻撃下では性能が大きく劣化することが確認された。

この結果は、実運用（SOC/CSIRT）において以下の示唆を与える。

---

## 1. False Negative のリスク

スパム検知における検知漏れは、
フィッシングやマルウェア感染に直結する。

精度（Accuracy）よりも、
Recallを重視した設計が現実的である。

---

## 2. False Positive の運用コスト

誤検知が多い場合：

- SOCアラート疲れ
- 重要アラートの見落とし
- 運用負荷増大

が発生する。

そのため単純な閾値引き下げではなく、
業務影響を考慮したチューニングが必要。

---

## 3. モデル更新戦略

攻撃手法は継続的に変化するため：

- 定期再学習
- データドリフト監視
- 継続的評価

が不可欠。

MLモデルは「作って終わり」ではなく、
継続的運用が前提である。

---

## 4. 実務的示唆

本PoCから、AIセキュリティでは：

- 精度最適化だけでは不十分
- 脅威モデリングに基づくロバスト性評価が必須
- 運用設計とセットで検討すべき

ということが示された。

今後は adversarial training や ensemble による
更なる耐攻撃性向上を検討する。


# Conclusion

- ML検知器は攻撃下で大きく性能劣化する
- 精度よりロバスト性が重要
- 脅威モデリング＋継続的評価が不可欠

AIセキュリティでは、
単なるモデル構築ではなく「攻撃を前提とした設計」が必要である。
